In [ ]:
# Dependencies for Selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd

# Import external file
from odds_scrapper import scrapper
from name_translator import companies, seria_a

In [ ]:
# Leagues to be scrapped
league_dict = {
    "Seria-A": {
        "Season": "2018-2019",
        "name_translator": seria_a,
        "url": "http://info.310win.com/cn/League/2018-2019/34.html/",
        "team_ct": 20
    }
}

In [ ]:
# Loop through leagues to scrape odds data
for key in list(league_dict.keys()):
    
    # Parameters for "scrapper" function
    season = league_dict[key]["Season"]
    name_translator = league_dict[key]["name_translator"]
    url = league_dict[key]["url"]
    team_ct = league_dict[key]["team_ct"]

    # Scrape data
    scrapper(key, season, name_translator, url, team_ct)